In [39]:
in_scope_window.loc[:, "window_delta"] = (
    in_scope_window["timestamp"] - in_scope_window["timestamp"].min()
).dt.total_seconds() + 1

In [40]:
# Because of data quality issue

in_scope_window_size = (
    min([in_scope_window["timestamp"].max(), complete_data_present_till]) -
    in_scope_window["timestamp"].min()
).total_seconds()
if in_scope_window_size < 1:
    in_scope_window_size = 1

85500

In [41]:
in_scope_edges = pov_df.groupby(["source", "target"]).agg(is_laundering=("is_laundering", "max")).reset_index()

pos_all = in_scope_edges[in_scope_edges["is_laundering"] == 1]
neg_all = in_scope_edges[in_scope_edges["is_laundering"] == 0]
num_neg_sources = neg_all["source"].nunique()
num_selected_neg_sources = int(np.ceil(num_neg_sources * SAMPLE_SIZE_NEG_SOURCES))
if num_selected_neg_sources > num_neg_sources:
    num_selected_neg_sources = int(num_neg_sources)
selected_neg_sources = np.random.choice(neg_all["source"].unique(), num_selected_neg_sources, replace=False)
in_scope_sources = set(selected_neg_sources).union(pos_all["source"])

in_scope_edges = in_scope_edges[in_scope_edges["source"].isin(in_scope_sources)].reset_index(drop=True)
in_scope_nodes = list(set(in_scope_edges["source"]).union(in_scope_edges["target"]))

In [42]:
def aggregate_edges(data_input):
    data_aggregated = (
        data_input.groupby(["source", "target"])
        .agg(
            amount=("amount", "sum")
        )
    ).reset_index()

    source_totals = (
        data_aggregated.groupby("source")
        .agg({"amount": "sum"})["amount"]
        .to_dict()
    )
    target_totals = (
        data_aggregated.groupby("target")
        .agg({"amount": "sum"})["amount"]
        .to_dict()
    )

    data_aggregated.loc[:, "total_sent_by_source"] = data_aggregated.loc[
        :, "source"
    ].apply(lambda x: source_totals[x])
    data_aggregated.loc[:, "total_received_by_target"] = data_aggregated.loc[
        :, "target"
    ].apply(lambda x: target_totals[x])
    data_aggregated.loc[:, "weight"] = data_aggregated.apply(
        lambda x: (
            (x["amount"] / x["total_sent_by_source"])
            + (x["amount"] / x["total_received_by_target"])
        ),
        axis=1,
    )
    return data_aggregated.loc[:, ["source", "target", "weight"]]

In [43]:
window_edges = aggregate_edges(in_scope_window)

In [44]:
graph = ig.Graph.DataFrame(window_edges, use_vids=False, directed=True)

In [45]:
communities = get_communities_multi_proc(in_scope_nodes, graph, NUM_PROCS)
sizes = [len(x[1]) for x in communities]
round(np.mean(sizes)), round(np.max(sizes)), sum(sizes)

CPU times: user 414 ms, sys: 96.1 ms, total: 510 ms
Wall time: 15.6 s


(11, 75, 755713)

In [46]:
in_scope_window.loc[:, "edge"] = in_scope_window.apply(
    lambda x: tuple(sorted([x["source"], x["target"]])), axis=1
)
in_scope_window.loc[:, "edge"] = in_scope_window.loc[:, "edge"].apply(
    lambda x: f"{x[0]}-{x[1]}"
)
in_scope_window.set_index("edge", inplace=True)

In [47]:
location_trx_comm = "transactions_communities"

In [48]:
shutil.rmtree(location_trx_comm, ignore_errors=True)
os.mkdir(location_trx_comm)

shutil.rmtree(location_trx_comm, ignore_errors=True)
os.mkdir(location_trx_comm)

communities = dict(communities)
communities_keys = [x for x in communities.keys()]

chunk_size = int(np.ceil(len(communities_keys) / NUM_PROCS))

number_of_chunks = int(np.ceil(len(communities_keys) / chunk_size))
chunks = np.array_split(communities_keys, number_of_chunks)
for index, chunk in enumerate(chunks):
    comm_inner = []
    for key in chunk:
        comm_node = communities[key]
        comm_inner += [[key, tuple(sorted(x))] for x in combinations(comm_node, 2)]
    edge_combinations = pd.DataFrame(comm_inner, columns=["id", "edge"])
    edge_combinations.loc[:, "edge"] = edge_combinations.loc[:, "edge"].apply(
        lambda x: f"{x[0]}-{x[1]}"
    )
    edge_combinations.set_index("edge", inplace=True)
    edge_combinations.join(in_scope_window, how="inner").reset_index(drop=True).to_parquet(
        f"{location_trx_comm}/part-{index}.parquet"
    )
    if not (index % 5):
        pass  # print(index, len(chunks))

0 8
5 8
CPU times: user 15.1 s, sys: 350 ms, total: 15.5 s
Wall time: 15.5 s


In [50]:
features = get_features_multi_proc(
    glob(f"{location_trx_comm}{os.sep}*"), "features.get_features_chunk_with_gf",
    reset_staging=True
)
features.loc[:, "in_scope_window_size"] = in_scope_window_size

CPU times: user 83 ms, sys: 250 ms, total: 333 ms
Wall time: 1min 5s


In [51]:
in_scope_trx_communities = in_scope_window.set_index("source").join(
    pd.DataFrame(index=in_scope_nodes), how="inner"
).reset_index()

In [52]:
iterator_chunk_as_pickles, _ = create_workload_for_multi_proc(
    in_scope_trx_communities["source"].nunique(),
    in_scope_trx_communities.groupby("source"), 
    NUM_PROCS, shuffle=False
)
features_source = get_features_multi_proc(
    iterator_chunk_as_pickles, "features.get_features_chunk_without_gf",
)

CPU times: user 2.86 s, sys: 221 ms, total: 3.08 s
Wall time: 38.3 s


In [ ]:
iterator_chunk_as_pickles, _ = create_workload_for_multi_proc(
    in_scope_trx_communities["target"].nunique(),
    in_scope_trx_communities.groupby("target"), 
    NUM_PROCS, shuffle=False
)
features_target = get_features_multi_proc(
    iterator_chunk_as_pickles, "features.get_features_chunk_without_gf",
)

In [ ]:
if not pov_features_generated:
    pov_features_df = []
    for k, v in pov_df[pov_df["source"].isin(in_scope_sources)].groupby(
        ["source", "target"]
    ):
        pov_features_df.append(pov_features(k, v))
    pov_features_df = pd.DataFrame(pov_features_df)

In [ ]:
def reset_index(df_input, index_name):
    df_input.index.name = index_name
    df_input = df_input.reset_index()
    return df_input

In [ ]:
if not pov_features_generated:
    all_features = pov_features_df

all_features = all_features.set_index("source").join(
    features.set_index("key"), how="left", rsuffix=f"_{window_name}_source"
)
all_features = reset_index(all_features, "source")

all_features = all_features.set_index("target").join(
    features.set_index("key"), how="left", rsuffix=f"_{window_name}_target"
)
all_features = reset_index(all_features, "target")

all_features = all_features.set_index("source").join(
    features_source.set_index("key"), how="left", rsuffix=f"_{window_name}_1_hop_for_source_as_source"
)
all_features = reset_index(all_features, "source") 

all_features = all_features.set_index("target").join(
    features_source.set_index("key"), how="left", rsuffix=f"_{window_name}_1_hop_for_source_as_target"
)
all_features = reset_index(all_features, "target")      

all_features = all_features.set_index("source").join(
    features_target.set_index("key"), how="left", rsuffix=f"_{window_name}_1_hop_for_target_as_source"
)
all_features = reset_index(all_features, "source") 

all_features = all_features.set_index("target").join(
    features_target.set_index("key"), how="left", rsuffix=f"_{window_name}_1_hop_for_target_as_target"
)
all_features = reset_index(all_features, "target")  

pov_features_generated = True